# Importing necessary libraries

In [15]:
import cv2
import numpy as np
import os
import pandas as pd

# Fetching Input for the FP input 

In [16]:
inp_fp = cv2.imread("FPDB/100__M_Left_index_finger.bmp") #read the fingerprint input image
cv2.imshow("Fingerprint Input", cv2.resize(inp_fp, None, fx=3, fy=3)) #show image in a different window
cv2.waitKey(0) #wait for close on output window
cv2.destroyAllWindows() #after closing,delete the window

# Fingerprint match using SIFT 

In [17]:
def fp_match(fpin):
    for file in [file for file in os.listdir("FPDB")]: #traverse through the database for matching fingerprint
        fingerprint_database_image = cv2.imread('FPDB/'+file) #reading the traversed file
        #SIFT-->SCALE INVARIANT FEATURE TRANSFORM
        #before SIFT,we need to create a SIFT feature detector object

        #sift = cv2.xfeatures2d.SIFT_create()  for OpenCV 3.4.1
        sift=cv2.SIFT_create()  #for OpenCV 2
                                                                            #FINDING KEYPOINTS(Detect) AND DESCRIPTORS(Compute)
        keypoints_1, descriptors_1 = sift.detectAndCompute(fpin, None) #in the original image 

        keypoints_2, descriptors_2 = sift.detectAndCompute(fingerprint_database_image, None) #in the traversed image

        #Matching and comparing both images using the Keypoints
        #FLANN-Fast Library for Approximate Nearest Neighbors Based Matcher 
        #is used for the comparision of the both,which matches features of both finger prints
        #quick and efficient
        #uses Clustering and Search in MultiDimensional Spaces

        #knnMatch to implement Image matching and sort the match results
        #k=2-->best 2 matches for each keypoint
        #matches = cv2.FlannBasedMatcher(dict(algorithm=FLANN_INDEX_KDTREE, trees=5),dict()).knnMatch(descriptors_1, descriptors_2, k=2)

        matches = cv2.FlannBasedMatcher(dict(algorithm=1, trees=10),dict()).knnMatch(descriptors_1, descriptors_2, k=2)
        match_points = []
        #print(matches)
        for p, q in matches:
            #CHECKING FOR GOOD MATCHES
            if p.distance < 0.1*q.distance:
                match_points.append(p)
            keypoints = 0
            if len(keypoints_1) <= len(keypoints_2):
                keypoints = len(keypoints_1)            
            else:
                keypoints = len(keypoints_2)

            #print(keypoints)

            #if we have a proper match,show the output of matched file,%match and the matching lines
            if (len(match_points) / keypoints)>0.95:
                matchper=round(len(match_points) / keypoints * 100,2) #Successful matches from the features tracked
                matchfromdb=str(file) 
                #result = cv2.drawMatches(test_original, keypoints_1, fingerprint_database_image, keypoints_2, match_points, None) 
                #result = cv2.resize(result, None, fx=5.5, fy=5.5)
                #cv2.imshow("result", result)
                #cv2.waitKey(0)
                #cv2.destroyAllWindows()
                return matchper,matchfromdb

# Performing fp recognition on the input fp

In [18]:
matchacc,fname=fp_match(inp_fp)

# A sample DB to enhance the Interface!

In [19]:
DB=pd.read_csv('Group11-DB.csv',index_col="Employee ID")
DB

,Name
Employee ID,
100,Mira Sanford
101,Kayley Strickland
102,Christina Frazier
103,Derick Whitney
104,Colin Fields
105,Brenna Graham
106,Mylie Espinoza
107,Zaria Rivas
108,Johnathon Gilbert


# Displaying output

In [20]:
lst=fname.split("_")
Name_from_csv=DB.loc[int(lst[0])]["Name"]
print("Welcome",Name_from_csv+"!")
print("Employee ID:",lst[0])
print("Finger used:",lst[3],lst[4],"finger ("+str(matchacc)+"% match)")

Welcome Mira Sanford!
Employee ID: 100
Finger used: Left index finger (97.22% match)
